1.	What is a neural network? What are the general steps required to build a neural network?

A neural network is an artifical network based on a brain network. A neural network has many layers where the first layer is the input layer and the last layer is the output layer. Each layer, from the first layer, feeds into the next layer. The general steps to building a neural network is to identify input and target values from your dataset. Identify the number of hidden layers, train your neural network, activation of the neural network using a model that goes best with what you're trying to predict. Finally, testing the neural network to see if it's a good fit. 

2.	Generally, how do you check the performance of a neural network? Why? 

Check the performance of a neural network using RMSE, MSE, Mean absolute error, accuracy, and precision and recall. These all tell your how well the neural network predicts the target variable. Based on the performance metric, the neural network can be tuned to make it perform better. 

In [2]:
import pandas as pd
import numpy as np

abalone_df = pd.read_table("../week_17/abalone.data", sep=",",header=None)
abalone_df.head()

,0,1,2,3,4,5,6,7,8
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


In [3]:
abalone_df.columns=["Sex","Length","Diameter","Height","Whole_wt","Shucked_wt","Viscera_wt","Shell_wt","Rings"]
#to predict age add 1.5
abalone_df["Age"] = abalone_df["Rings"] + 1.5
abalone_df.head()

,Sex,Length,Diameter,Height,Whole_wt,Shucked_wt,Viscera_wt,Shell_wt,Rings,Age
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15,16.5
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7,8.5
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9,10.5
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10,11.5
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7,8.5


In [4]:
#Encode Sex column using one hot encoding
dum_df = pd.get_dummies(abalone_df['Sex'])
abalone_df = abalone_df.join(dum_df)
abalone_df.head()

,Sex,Length,Diameter,Height,Whole_wt,Shucked_wt,Viscera_wt,Shell_wt,Rings,Age,F,I,M
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15,16.5,0,0,1
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7,8.5,0,0,1
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9,10.5,1,0,0
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10,11.5,0,0,1
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7,8.5,0,1,0


In [5]:
#drop Sex column after one hot encoding
abalone_df = abalone_df.drop('Sex',axis=1)

In [6]:
abalone_df.describe()

,Length,Diameter,Height,Whole_wt,Shucked_wt,Viscera_wt,Shell_wt,Rings,Age,F,I,M
count,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000
mean,0.523992,0.407881,0.139516,0.828742,0.359367,0.180594,0.238831,9.933684,11.433684,0.312904,0.321283,0.365813
std,0.120093,0.099240,0.041827,0.490389,0.221963,0.109614,0.139203,3.224169,3.224169,0.463731,0.467025,0.481715
min,0.075000,0.055000,0.000000,0.002000,0.001000,0.000500,0.001500,1.000000,2.500000,0.000000,0.000000,0.000000
25%,0.450000,0.350000,0.115000,0.441500,0.186000,0.093500,0.130000,8.000000,9.500000,0.000000,0.000000,0.000000
50%,0.545000,0.425000,0.140000,0.799500,0.336000,0.171000,0.234000,9.000000,10.500000,0.000000,0.000000,0.000000
75%,0.615000,0.480000,0.165000,1.153000,0.502000,0.253000,0.329000,11.000000,12.500000,1.000000,1.000000,1.000000
max,0.815000,0.650000,1.130000,2.825500,1.488000,0.760000,1.005000,29.000000,30.500000,1.000000,1.000000,1.000000


In [7]:
#remove outliers
Q1 = abalone_df.quantile(0.25)
Q3 = abalone_df.quantile(0.75)
IQR = Q3 - Q1

abalone_df = abalone_df[~((abalone_df < (Q1 - 1.5 * IQR)) |(abalone_df > (Q3 + 1.5 * IQR))).any(axis=1)]
abalone_df.shape

(3781, 12)

In [8]:
# Import `Sequential` from `keras.models`
from keras.models import Sequential

# Import `Dense` from `keras.layers`
from keras.layers import Dense

X = abalone_df.drop(['Age','Rings'],axis=1)
y = abalone_df['Age']

In [15]:
from sklearn.model_selection import train_test_split

# Split into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=1)

In [23]:
# define the keras model
model = Sequential()
model.add(Dense(10, input_dim=10, activation='relu'))
model.add(Dense(1, activation='linear'))
#model.add(Dense(1, activation='sigmoid'))

In [26]:
# compile the keras model
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae','RootMeanSquaredError'])

In [27]:
# fit the keras model on the dataset
model.fit(X_train, y_train, epochs=100, batch_size=10)

Epoch 1/100
265/265 [==============================] - 1s 1ms/step - loss: 111.2902 - mae: 10.2758 - root_mean_squared_error: 10.5379
Epoch 2/100
265/265 [==============================] - 0s 1ms/step - loss: 30.6196 - mae: 5.0843 - root_mean_squared_error: 5.4950
Epoch 3/100
265/265 [==============================] - 0s 2ms/step - loss: 5.0649 - mae: 1.7129 - root_mean_squared_error: 2.2499
Epoch 4/100
265/265 [==============================] - 0s 1ms/step - loss: 4.5475 - mae: 1.6290 - root_mean_squared_error: 2.1316
Epoch 5/100
265/265 [==============================] - 0s 1ms/step - loss: 4.0598 - mae: 1.5247 - root_mean_squared_error: 2.0132
Epoch 6/100
265/265 [==============================] - 0s 1ms/step - loss: 3.7427 - mae: 1.4879 - root_mean_squared_error: 1.9338
Epoch 7/100
265/265 [==============================] - 0s 1ms/step - loss: 3.6500 - mae: 1.4663 - root_mean_squared_error: 1.9097
Epoch 8/100
265/265 [==============================] - 1s 2ms/step - loss: 3.6047 - m

265/265 [==============================] - 0s 1ms/step - loss: 2.4436 - mae: 1.2054 - root_mean_squared_error: 1.5620
Epoch 64/100
265/265 [==============================] - 0s 1ms/step - loss: 2.5581 - mae: 1.2307 - root_mean_squared_error: 1.5988
Epoch 65/100
265/265 [==============================] - 0s 1ms/step - loss: 2.7065 - mae: 1.2858 - root_mean_squared_error: 1.6434
Epoch 66/100
265/265 [==============================] - 0s 2ms/step - loss: 2.6010 - mae: 1.2509 - root_mean_squared_error: 1.6111
Epoch 67/100
265/265 [==============================] - 0s 2ms/step - loss: 2.4771 - mae: 1.2209 - root_mean_squared_error: 1.5726
Epoch 68/100
265/265 [==============================] - 0s 2ms/step - loss: 2.6171 - mae: 1.2313 - root_mean_squared_error: 1.6168
Epoch 69/100
265/265 [==============================] - 0s 2ms/step - loss: 2.5287 - mae: 1.2433 - root_mean_squared_error: 1.5897
Epoch 70/100
265/265 [==============================] - 0s 2ms/step - loss: 2.3296 - mae: 1.1935

4.	Write another algorithm to predict the same result as the previous question using either KNN or logistic regression.

In [29]:
from sklearn.metrics import mean_squared_error as MSE
from sklearn.neighbors import KNeighborsRegressor

#KNN model - using regressor
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(X_train,y_train)
y_predict = knn.predict(X_test)

knn_mse = MSE(y_test,y_predict)
knn_rmse = knn_mse ** (1/2)

print("Mean Squared Error: ", knn_mse)
print("Root Mean Squred Error: ", knn_rmse)

Mean Squared Error:  3.200528634361233
Root Mean Squred Error:  1.7890021336938737


5.	Create a neural network using pytorch to predict the same result as question 3. 

In [71]:
import torch
import torch.nn as nn
import torch.nn.functional as F #this has activation functions

X = abalone_df.drop(['Age','Rings'],axis=1).values
y = abalone_df['Age'].values

# Split into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=1)

# Creating tensors
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)

y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

y_train = y_train.float()
y_train = y_train.view(-1,1)

y_test = y_test.float()
y_test = y_test.view(-1,1)

In [72]:
class ANN_Model(nn.Module):
    def __init__(self, input_features=10, hidden1=20, hidden2=20, out_features =2):
        super().__init__()
        self.layer_1_connection = nn.Linear(input_features, hidden1)
        self.layer_2_connection = nn.Linear(hidden1, hidden2)
        self.out = nn.Linear(hidden2, out_features)
    
    def forward(self, x):
        #apply activation functions
        x = F.relu(self.layer_1_connection(x))
        x = F.relu(self.layer_2_connection(x))
        x = self.out(x)
        return x

In [73]:
torch.manual_seed(42)

#instantiate the model
model = ANN_Model()

In [74]:
# loss function
loss_function = nn.MSELoss()

#optimizer
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

In [75]:
#run model through multiple epochs/iterations
final_loss = []
n_epochs = 500
for epoch in range(n_epochs):
    y_pred = model.forward(X_train)
    loss = loss_function(y_pred, y_train)
    final_loss.append(loss)
    
    if epoch % 10 == 1:
        print(f'Epoch number: {epoch} with loss: {loss.item()}')
    
    optimizer.zero_grad() #zero the gradient before running backwards propagation
    loss.backward() #for backward propagation 
    optimizer.step() #performs one optimization step each epoch

Epoch number: 1 with loss: 109.35504150390625
Epoch number: 11 with loss: 87.32950592041016
Epoch number: 21 with loss: 28.671648025512695
Epoch number: 31 with loss: 13.375268936157227
Epoch number: 41 with loss: 4.560524940490723
Epoch number: 51 with loss: 5.384434700012207
Epoch number: 61 with loss: 4.519216537475586
Epoch number: 71 with loss: 4.03368616104126
Epoch number: 81 with loss: 3.947730302810669
Epoch number: 91 with loss: 3.8341777324676514
Epoch number: 101 with loss: 3.720399856567383
Epoch number: 111 with loss: 3.6262707710266113
Epoch number: 121 with loss: 3.545247793197632
Epoch number: 131 with loss: 3.472867727279663
Epoch number: 141 with loss: 3.408388376235962
Epoch number: 151 with loss: 3.3517091274261475
Epoch number: 161 with loss: 3.3024284839630127
Epoch number: 171 with loss: 3.260009288787842
Epoch number: 181 with loss: 3.2238121032714844
Epoch number: 191 with loss: 3.193145990371704
Epoch number: 201 with loss: 3.1672816276550293
Epoch number: 21

The keras model performed better. With Keras, I obtained a low RMSE of 1.51. With KNN, my lowest RMSE was 1.79, and with pytorch, it was 1.68. Keras probably performaed better because of the ability to add multiple hidden layers to make the model perform better. 